In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import LineString
from shapely import wkt
import numpy as np
import swifter
from generator.preprocess import *
from generator.road_network import RoadNetwork

In [2]:
# load mapped dataset
df = pd.read_csv("../datasets/trajectories/Porto/mapped.csv")
# load network
network = RoadNetwork("Porto, Portugal", network_type="drive", retain_all=True, truncate_by_edge=True)

In [3]:
cdf = clip_trajectories(df, network.bounds)

Pandas Apply:   0%|          | 0/1641024 [00:00<?, ?it/s]

In [5]:
fdf = filter_min_points(cdf, 10)

Dask Apply:   0%|          | 0/32 [00:00<?, ?it/s]

In [18]:
# save both dataframes
fdf.to_csv("../datasets/trajectories/Porto/clipped_porto_full_10mil.csv")